In [ ]:
!pip install halo
!pip install numerapi
!pip install catboost

import numpy as np
import pandas as pd
import scipy
from halo import Halo
from pathlib import Path
import json
from scipy.stats import skew
import numerapi
import gc
from numerapi import NumerAPI
import time


from catboost import CatBoostRegressor
import catboost


from utils_plus import *

#download_data()

In [ ]:
start_time = time.time()

model_name = "catboost2_small"

with open("./data/features.json", "r") as f:
    feature_metadata = json.load(f)
features = feature_metadata["feature_sets"]["fncv3_features"]

# read the training and validation data given the predefined features stored in parquets as pandas DataFrames
training_data, validation_data = read_learning_data(features)
# extract feature matrix and target vector used for training
X_train = training_data.filter(like='feature_', axis='columns')
y_train = training_data[TARGET_COL]
# extract feature matrix and target vector used for validation
X_val = validation_data.filter(like='feature_', axis='columns')
y_val = validation_data[TARGET_COL]
# "garbage collection" (gc) gets rid of unused data and frees up memory
gc.collect()


In [ ]:
num_feature_neutralization = 40  # parameter for feature neutralization used after we make our predictions
params ={"n_estimators": 11000,
          "learning_rate": 0.003,
          "task_type":"GPU",
          "depth": 9,
          'l2_leaf_reg': 9}

model = CatBoostRegressor(**params)


spinner.start('Training model')
model.fit(X_train, y_train, early_stopping_rounds = 10, verbose = False)
spinner.succeed()
gc.collect()
spinner.start('Saving model')
model.save_model(f'models/{model_name}.txt')
spinner.succeed()


In [10]:
######## Delete belowwhen done using diagnostics tool
validation_data.loc[:, f"preds_{model_name}"] = model.predict(X_val)
spinner.succeed()
gc.collect()
neutralize_riskiest_features(training_data, validation_data, features, model_name, k=num_feature_neutralization)

print('Exporting Predictions to csv...')
validation_data["prediction"] = validation_data[f"preds_{model_name}_neutral_riskiest_{num_feature_neutralization}"] \
    .rank(pct=True)
validation_data["prediction"].to_csv(f"predictions/{model_name}.csv")



end_time = time.time()
time_elapsed_mins = (end_time - start_time)//60
time_elapsed_secs = (end_time - start_time)%60
print(f"Time elapsed: {time_elapsed_mins} mins {time_elapsed_secs} secs")



print('Done!')

✔ Saving model
Exporting Predictions to csv...
Time elapsed: 5.0 mins 54.7894024848938 secs
Done!
Error in callback <function Halo.__init__.<locals>.clean_up at 0x7f5f0134d790> (for post_run_cell), with arguments args (<ExecutionResult object at 7f5f0132d190, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f5f0132d220, raw_cell="######## Delete belowwhen done using diagnostics t.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/gordontan/Desktop/numerai/yup.ipynb#W3sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: clean_up() takes 0 positional arguments but 1 was given